In [20]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import math
import numpy as np

In [21]:
import pickle
with open('pkls/m3_item.pkl', "r") as f:
    df_m3_item_load = pickle.load(f)
    print(df_m3_item_load)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [22]:
print(np.load('mat_m3_item.npy'))

[[ 5  2  0 ...,  1  0  0]
 [12  1  0 ...,  0  0  1]
 [ 1  0  0 ...,  0  0  0]
 ..., 
 [ 8  9  0 ...,  1  0  0]
 [ 0  0  0 ...,  0  0  0]
 [ 5  6  0 ...,  0  0  0]]


In [2]:
class MF(nn.Module):
    def __init__(self, input_size, K): 
        super(MF,self).__init__()
        self.input_size = input_size
        self.P = nn.Parameter(torch.Tensor(K, input_size[0]).uniform_(0,1))
        self.Q = nn.Parameter(torch.Tensor(K, input_size[1]).uniform_(0,1))
        
    def forward(self):
        return torch.matmul(torch.t(self.P), self.Q) 

    def project(self):
        self.P.data = torch.max(self.P.data, torch.zeros(self.P.data.size()))
        self.Q.data = torch.max(self.Q.data, torch.zeros(self.Q.data.size()))

In [3]:
def ll_gaussian(x, mu, sigma2, missing=False):
    if missing:
        weight = (x != 0).float()
        return -0.5 * (math.log(2 * math.pi * sigma2) + torch.pow(x - mu, 2) / sigma2) * weight
    return -0.5 * (math.log(2 * math.pi * sigma2) + torch.pow(x - mu, 2) / sigma2)

In [4]:
def ll_poisson(x, lam, missing=False):
    if missing:
        weight = (x != 0).float()
        return (x * torch.log(lam) - lam) * weight
    return x * torch.log(lam) - lam

In [5]:
def update_param_gaussian(R, model, optimizer, missing=False):
    optimizer.zero_grad()
    pred = model()
    log_likelihood = ll_gaussian(R, pred, 1, missing=missing)
    prior_P = ll_gaussian(model.P, 0, 1/0.003)
    prior_Q = ll_gaussian(model.Q, 0, 1/0.003)
    loss = -(torch.sum(log_likelihood) + torch.sum(prior_P) + torch.sum(prior_Q))
    loss.backward()
    optimizer.step()
    return loss.data[0]

In [6]:
def update_param_poisson(R, model, optimizer, missing=False):
    optimizer.zero_grad()
    pred = model()
    log_likelihood = ll_poisson(R, pred, missing=missing)
    loss = -torch.sum(log_likelihood)
    loss.backward()
    optimizer.step()
    return loss.data[0]


In [7]:
if __name__ == '__main__':
    R = Variable(torch.FloatTensor([
            [5, 3, 0, 1],
            [4, 0, 0, 1],
            [1, 1, 0, 5],
            [1, 0, 0, 4],
            [0, 1, 5, 4],
            ]
        ))

    model = MF(R.size(), 2)
    optimizer_p = optim.Adam([model.P], lr = 0.02)
    optimizer_q = optim.Adam([model.Q], lr = 0.02)

    for i in range(1000):
        update_param_poisson(R, model, optimizer_p, missing=True)
        model.project()
        loss = update_param_poisson(R, model, optimizer_q, missing=True)
        model.project()

        if i % 100 == 0:
            print("Epoch {}: Loss:{}".format(i+1, loss))

    print("Epoch {}: Loss:{}".format(i+1, loss))
    print("Ground Truth:{}".format(R.data))
    print("Predict:{}".format(model().data))

    print("P:{}".format(model.P.data))
    print("Q:{}".format(model.Q.data))


Epoch 1: Loss:51.99916076660156
Epoch 101: Loss:-2.2936182022094727
Epoch 201: Loss:-3.287109375
Epoch 301: Loss:-7.284323215484619
Epoch 401: Loss:-8.021031379699707
Epoch 501: Loss:-8.068026542663574
Epoch 601: Loss:-8.072629928588867
Epoch 701: Loss:-8.072924613952637
Epoch 801: Loss:-8.072936058044434
Epoch 901: Loss:-8.072937965393066
Epoch 1000: Loss:-8.072938919067383
Ground Truth:
 5  3  0  1
 4  0  0  1
 1  1  0  5
 1  0  0  4
 0  1  5  4
[torch.FloatTensor of size 5x4]

Predict:
 5.0000  3.0000  4.1549  1.0000
 4.0000  2.4167  3.5387  1.0000
 1.0000  1.0000  5.9859  5.0000
 1.0000  0.9167  4.9120  4.0000
 1.1429  1.0000  5.0000  4.0000
[torch.FloatTensor of size 5x4]

P:
 2.2320  1.7856  0.4464  0.4464  0.5102
 0.0000  0.1309  3.1421  2.4875  2.4688
[torch.FloatTensor of size 2x5]

Q:
 2.2401  1.3441  1.8615  0.4480
 0.0000  0.1273  1.6406  1.5276
[torch.FloatTensor of size 2x4]

